In [3]:

from jax import Array
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import seaborn as sns
import torch


from arc_prize.vis import COLORS

def visualize_embeddings(embeddings: Array, num_grids: int):
    """
    Visualize embeddings using t-SNE or PCA.
    
    :param embeddings: torch.Tensor of shape (n_samples, embedding_dim)
    :param labels: list of labels for each embedding (optional)
    :param method: 'tsne' or 'pca'
    """
    embeddings_np = embeddings
    
    
    tsne_reducer = TSNE(n_components=2, random_state=42)
    pca_reducer = PCA(n_components=2)
    
    reduced_tsne_embeddings = tsne_reducer.fit_transform(embeddings_np)
    reduced_pca_embeddings = pca_reducer.fit_transform(embeddings_np)
    
    cmap = mcolors.ListedColormap(COLORS)
    per_grid = len(reduced_tsne_embeddings) // num_grids
    grid_indices = torch.arange(num_grids).repeat_interleave(per_grid)
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

    scatter1 = ax1.scatter(reduced_tsne_embeddings[:, 0], reduced_tsne_embeddings[:, 1], c=grid_indices)
    plt.colorbar(scatter1, ax=ax1)
    ax1.set_title("Embeddings with TSNE by grid")

    scatter2 = ax2.scatter(reduced_pca_embeddings[:, 0], reduced_pca_embeddings[:, 1], c=grid_indices)
    plt.colorbar(scatter2, ax=ax2)
    ax2.set_title("Embeddings with PCA by grid")


    plt.show()

In [ ]:
from arc_prize.flax.models import create_arc_fixed_positional_encoding


enc = create_arc_fixed_positional_encoding(32, 10, 4)

print(enc.shape)

visualize_embeddings(enc.reshape(-1, 32), num_grids=9)



In [ ]:
from arc_prize.flax.models import ARCTransformerEncoderDecoderParams
from arc_prize.flax.train import TrainParams, train_and_evaluate_local

model_params = ARCTransformerEncoderDecoderParams(
  grid_dim=12,
  num_train_pairs=4,
  num_colors=10,
  num_encoder_layers=2,
  num_decoder_layers=2,
  num_heads=2,
  d_model=16,
  d_ff=16*4,
  dropout=0.1
)

train_params = TrainParams(
  batch_size=20,
  learning_rate=1e-4,
  weight_decay=1e-4,
  warmup_steps=5,
  train_steps_per_epoch=10,
  eval_steps_per_epoch=5,
  dataset_dirs=["/Users/pfh/work/arc-data/flip"],
  loss_class_weights={0: 0.2}
)

model_dir = "/Users/pfh/work/arc-models/flax-2"
num_epochs = 100

train_and_evaluate_local(model_dir, num_epochs)

In [ ]:
from arc_prize.flax.models import ARCTransformerEncoderDecoderParams
from arc_prize.flax.train import TrainParams
import modal
import petname

model_params = ARCTransformerEncoderDecoderParams(
  grid_dim=12,
  num_train_pairs=4,
  num_colors=10,
  num_encoder_layers=2,
  num_decoder_layers=2,
  num_heads=4,
  d_model=64,
  d_ff=64*4,
  dropout=0.1
)

train_params = TrainParams(
  batch_size=20,
  learning_rate=1e-4,
  weight_decay=1e-4,
  warmup_steps=50,
  train_steps_per_epoch=30,
  eval_steps_per_epoch=5,
  dataset_dirs=["/vol/data/flip"],
  loss_class_weights={0: 0.2}
)

model_name = petname.generate(words=3, separator='_')
model_dir = f"/vol/models/{model_name}"
num_epochs = 100


fn = modal.Function.lookup("arc-jax", "train")
fn_call = fn.spawn(model_dir, train_params, model_params, num_epochs)
print("Model name", model_name, fn_call.object_id)


In [ ]:
from flax import nnx

rng1 = nnx.Rngs(0, params=1)
rng2 = nnx.Rngs(0)
print(rng1, rng2)